# Fuel Weight

This section briefly describes the code used for computing the 

In [ ]:
import numpy as np
from parameters import AircraftParameters

ImportError: attempted relative import with no known parent package

In [ ]:
import numpy as np
from parameters import AircraftParameters

class Aircraft():

    def __init__(self, parameters: AircraftParameters):
        """
            Class defining the aircraft and the methods for estimating weights
        """

        self.parameters = parameters

        self.parameters.b = (self.parameters.A * self.parameters.S)**0.5
        self.parameters.e = 1.78 * (1 - 0.045*self.parameters.A**0.68) - 0.64

        # Compute wing planform
        taper_ratio = self.parameters.taper_ratio_wing
        self.parameters.wing_root_chord = 2*self.parameters.S / self.parameters.b / (1 + taper_ratio) # ft
        self.parameters.wing_tip_chord = taper_ratio * self.parameters.wing_root_chord # ft
        self.parameters.mac = 2/3 * self.parameters.wing_root_chord * (1 + taper_ratio + taper_ratio**2) / (1 + taper_ratio) # ft

        self.parameters.CD0 = self.compute_CD0()

    def compute_CD0(self):
        """
            Method to compute zero-lift drag
        """

        S = self.parameters.S
        M = self.parameters.cruise_mach_number
        tc = self.parameters.tc_avg
        xc = 0.3
        sweep = self.parameters.sweep_qc
        Swet_wing = 1.7 * S

        Re = self.parameters.rho_cruise * self.parameters.cruise_speed * self.parameters.mac / self.parameters.dyn_viscosity
        Cf_wing = 0.455/np.log10(Re)**2.58 

        ff_wing = (1 + 0.6*tc/xc + 100*tc**4) * (1.34*M**0.18*np.cos(np.radians(sweep))**0.28)
        Q_wing = 1.0

        CD0_wing = 1.05 * Cf_wing * ff_wing * Q_wing * Swet_wing / S

        CDO_remaining = 0.024899

        CD0 = CDO_remaining + CD0_wing

        return CD0

    def compute_fuel_weight(self, weight):
        """
            Method to compute fuel weight fraction for a given maximum takeoff weight

            This function essentially performs mission analysis

            Parameters
            ----------
            weight: float
                maximum takeoff weight for which fuel fraction needs to be computed
        """

        R = self.parameters.range
        cruise_speed = self.parameters.cruise_speed
        W1_W0 = self.parameters.W1_W0
        W2_W1 = self.parameters.W2_W1
        W4_W3 = self.parameters.W4_W3

        # Compute cruise weight fraction
        weight = W2_W1 * W1_W0 * weight # weight at the start of cruise
        W3_W2, avg_cruise_power, avg_cruise_CL = self.compute_cruise_weight_fraction(weight, R, cruise_speed)
        self.parameters.W3_W2 = W3_W2
        self.parameters.avg_cruise_power = avg_cruise_power
        self.parameters.avg_cruise_CL = avg_cruise_CL

        # Compute loiter weight fraction
        W5_W4 = self.parameters.W5_W4
        weight = W5_W4 * W4_W3 * W3_W2 * weight # weight at the start of loiter
        W6_W5, avg_loiter_power, avg_loiter_speed = self.compute_loiter_weight_fraction(weight, R, cruise_speed)
        self.parameters.W6_W5 = W6_W5
        self.parameters.avg_loiter_power = avg_loiter_power
        self.parameters.avg_loiter_speed = avg_loiter_speed

        W7_W6 = self.parameters.W7_W6
        W8_W7 = self.parameters.W8_W7

        # compute final weight fraction
        W8_W0 = W8_W7 * W7_W6 * W6_W5 * W5_W4 * W4_W3 * W3_W2 * W2_W1 * W1_W0 # mission weight fraction
        Wf_W0 = 1.06 * (1 - W8_W0) # 6% is for trapped unusable fuel

        self.parameters.fuel_weight = Wf_W0 * weight

        return Wf_W0 # fuel fraction

    def compute_cruise_weight_fraction(self, weight, R, cruise_speed, num_segements=100):
        """
            Compute the weight fraction for cruise segment

            Parameters
            ----------
            weight : float
                Initial weight at the beginning of cruise (lbs)
            R : float
                Range of cruise (nm)
            cruise_speed : float
                speed during cruise (knots)
            num_segements : int
                Number of segments to divide the cruise into
            
            Returns
            -------
            cruise_weight_fraction: float
                Total weight fraction for the cruise segment
            avg_power_req: float
                Average power required during cruise (hp)
            avg_CL_cruise: float
                Average lift coefficient during cruise
        """

        # Parameters
        rho = self.parameters.rho_cruise
        CD0 = self.parameters.CD0
        e = self.parameters.e
        PI = self.parameters.PI
        Cbhp = self.parameters.Cbhp
        install_loss_factor = self.parameters.install_loss_factor
        prop_eff_cruise = self.parameters.prop_eff_cruise

        segment_range = R / num_segements # segment range, nmi
        segment_time = segment_range / cruise_speed # hr
        cruise_speed = cruise_speed * 1.68781  # Convert to ft/s
        q = 0.5 * rho * cruise_speed**2 # lbs/sq ft
        K = 1/PI/A/e

        # initialize empty lists
        cruise_fuel = 0
        power_req = []
        cruise_CL = []

        for i in range(num_segements):

            # Compute Cd
            CL = weight / q / S
            cruise_CL.append(CL)
            CD = CD0 + K * CL**2

            # Compute power required for this segment
            T = CD * q * S / install_loss_factor # lbs
            segment_power = T * cruise_speed / prop_eff_cruise / 550 # hp
            power_req.append(segment_power)
            
            # fuel spent 
            segment_fuel = Cbhp * segment_power * segment_time # lbs
            cruise_fuel += segment_fuel

            weight = weight - segment_fuel # weight at the end of the segment

        cruise_weight_fraction  = weight / (weight + cruise_fuel)
        avg_power_req = np.mean(power_req)
        avg_cruise_CL = np.mean(cruise_CL)

        return cruise_weight_fraction, avg_power_req, avg_cruise_CL

    def compute_loiter_weight_fraction(self, weight, E, num_segements=100):
        """
            Compute the weight fraction for loiter segment

            Parameters
            ----------
            weight : float
                Initial weight at the beginning of loiter (lbs)
            E : float
                loiter time (hr)
            num_segements : int
                Number of segments to divide the loiter into
            
            Returns
            -------
            weight_fraction: float
                Total weight fraction for the loiter segment
            avg_power_req: float
                Average power required during loiter (hp)
            avg_speed_loiter: float
                Average speed during loiter
        """

        # Parameters
        rho = self.parameters.rho_loiter
        CD0 = self.parameters.CD0
        e = self.parameters.e
        PI = self.parameters.PI
        Cbhp = self.parameters.Cbhp
        S = self.parameters.S
        install_loss_factor = self.parameters.install_loss_factor
        prop_eff_loiter = self.parameters.prop_eff_loiter

        segment_time = E / num_segements # hr

        K = 1/PI/A/e
        CL = (3*CD0/K)**0.5 # for max endurance for a prop airplane
        CD = CD0 + K * CL**2

        # initialize empty lists
        loiter_fuel = 0
        power_req = []
        loiter_speed = []

        for i in range(num_segements):

            V = ( weight/S * 2/rho * 1/CL )**0.5 # ft/s
            loiter_speed.append(V)
            q = 0.5 * rho * V**2 # lbs/sq ft

            # Compute power required for this segment
            T = CD * q * S / install_loss_factor # lbs
            segment_power = T * V / prop_eff_loiter / 550 # hp
            power_req.append(segment_power)

            # fuel spent 
            segment_fuel = Cbhp * segment_power * segment_time # lbs
            loiter_fuel += segment_fuel

            weight = weight - segment_fuel # weight at the end of the segment

        loiter_weight_fraction  = weight / (weight + loiter_fuel)
        avg_power_req = np.mean(power_req)
        avg_loiter_speed = np.mean(loiter_speed)
        
        return loiter_weight_fraction, avg_power_req, avg_loiter_speed

    def compute_empty_weight(self, MTOW):

        Wfw = self.parameters.fuel_weight
        Sw = self.parameters.S
        sweep = np.radians(self.parameters.sweep_qc)
        taper_ratio = self.parameters.taper_ratio_wing
        Nz = self.parameters.Nz
        tc = self.parameters.tc_avg

        wing_weight = 0.036*Sw**0.758 * Wfw**0.0035 * (A/np.cos(sweep)**2)**0.6 * q**0.006 * taper_ratio**0.04 \
        * (100*tc/np.cos(sweep))**-0.3 * (Nz*MTOW)**0.49

        pass

    def compute_MTOW(self):

        pass

    def compute_thrust(self):

        pass

    def compute_takeoff(self):

        pass

    def compute_landing(self):

        pass

    def compute_cruise_speed(self):

        pass

    def compute_climb_gradient(self):

        pass

In [ ]:
import numpy as np

In [1]:
import jdc

class TestABC():

    def __init__(self):

        print("Hello")

In [ ]:
%%add_to TestABC
def unique(self):

    print("Testing.,,,,")

In [ ]:
y = TestABC()

y.unique()

Testing.,,,,
